In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.special import softmax
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import math

In [2]:
recipes = pd.read_csv("../data/recipes/all_recipes.csv")
reviews = pd.read_csv("../data/users/all_users.csv")

In [3]:
transformed_user_ids = pd.DataFrame(reviews.user_id.unique()).reset_index()
transformed_user_ids.columns = ["transformed_user_id","user_id"]

In [4]:
transformed_recipe_ids = pd.DataFrame(reviews.recipe_id.unique()).reset_index()
transformed_recipe_ids.columns = ["transformed_recipe_id","recipe_id"]

In [5]:
reviews = reviews.merge(transformed_user_ids, on="user_id")
reviews = reviews.merge(transformed_recipe_ids, on="recipe_id")

In [6]:
# reviews["transformed_rating"] = reviews.rating - 1

In [7]:
user_vocab_size = reviews.transformed_user_id.max()
item_vocab_size = reviews.transformed_recipe_id.max()

In [8]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

item_embedding_layer = layers.Embedding(input_dim = item_vocab_size + 1, output_dim=128, input_length = None)
item_id = keras.Input(shape=(1,), name='item_ids')
item_embeddings = item_embedding_layer(item_id)
item_embeddings = layers.Dropout(rate=0.3)(item_embeddings)
item_embeddings = layers.Flatten()(item_embeddings)
item_embeddings = keras.Model(inputs=item_id, outputs=item_embeddings)

user_id = keras.Input(shape=(1,), name='user_ids')
user_embeddings = layers.Embedding(input_dim = user_vocab_size + 1, output_dim=128, input_length = None)(user_id)
user_embeddings = layers.Dropout(rate=0.3)(user_embeddings)
user_embeddings = layers.Flatten()(user_embeddings)
user_embeddings = keras.Model(inputs=user_id, outputs=user_embeddings)

dot = layers.Dot(axes=1)([user_embeddings.output, item_embeddings.output])
s = keras.Model(inputs=[user_id, item_id], outputs=dot)

combined = layers.concatenate([user_embeddings.output, item_embeddings.output])
z = layers.BatchNormalization(name="bn_top_0")(combined)
z = layers.Dense(32, activation='relu', name='top_combined_dense_1')(z)
z = layers.Dropout(rate=0.2)(z)
z = layers.BatchNormalization(name="bn_top_1")(z)
z = layers.Dense(16, activation='relu', name='top_combined_dense_2')(z)
z = layers.concatenate([z, s.output])
z = layers.Dense(1, name='predictions')(z)
model = keras.Model(inputs=[user_id, item_id], outputs=z)

In [9]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),  # Optimizer
              # Loss function to minimize
              loss=tf.keras.losses.MSE,  
              # List of metrics to monitor
              metrics= [keras.metrics.MeanSquaredError()])

In [10]:
train = reviews[reviews.date < "2018-01-01"]
test = reviews[reviews.date >= "2018-01-01"]

In [11]:
user_id_train = train.transformed_user_id[:-1000]
item_id_train = train.transformed_recipe_id[:-1000]
y_train = train.rating[:-1000]

user_id_test = test.transformed_user_id
item_id_test = test.transformed_recipe_id
y_test = test.rating

user_id_val = train.transformed_user_id[-1000:]
item_id_val = train.transformed_recipe_id[-1000:]
y_val = train.rating[-1000:]

In [13]:
keras.backend.clear_session()
# del model
history = model.fit([user_id_train, item_id_train], y_train,
                    batch_size=1000,
                    epochs=10,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=([user_id_val, item_id_val], y_val)
                   )
print('\nhistory dict:', history.history)

Epoch 1/10
54/54 [==============================] - 4s 67ms/step - loss: 1.2876 - mean_squared_error: 1.2876 - val_loss: 6.5862 - val_mean_squared_error: 6.5862
Epoch 2/10
54/54 [==============================] - 4s 66ms/step - loss: 0.8860 - mean_squared_error: 0.8860 - val_loss: 3.2589 - val_mean_squared_error: 3.2589
Epoch 3/10
54/54 [==============================] - 4s 68ms/step - loss: 0.6809 - mean_squared_error: 0.6809 - val_loss: 1.6457 - val_mean_squared_error: 1.6457
Epoch 4/10
54/54 [==============================] - 4s 67ms/step - loss: 0.5560 - mean_squared_error: 0.5560 - val_loss: 1.0411 - val_mean_squared_error: 1.0411
Epoch 5/10
54/54 [==============================] - 4s 70ms/step - loss: 0.4728 - mean_squared_error: 0.4728 - val_loss: 0.8594 - val_mean_squared_error: 0.8594
Epoch 6/10
54/54 [==============================] - 4s 70ms/step - loss: 0.3973 - mean_squared_error: 0.3973 - val_loss: 0.8233 - val_mean_squared_error: 0.8233
Epoch 7/10
54/54 [================

### Evaluation

#### Training metrics

In [14]:
predictions = model.predict([user_id_train, item_id_train])

In [15]:
print("train rmse:", math.sqrt(mean_squared_error(y_train, predictions)))

train rmse: 0.3818415760277013


#### Test metrics

In [16]:
predictions = model.predict([user_id_test, item_id_test])

In [17]:
print("test rmse:", math.sqrt(mean_squared_error(y_test, predictions)))

test rmse: 0.7569037349931316


### Adding content based features to the network

In [18]:
recipes = recipes[["recipe_id","title"]]

In [19]:
train = train.merge(recipes, how="inner", on="recipe_id")
test = test.merge(recipes, how="inner", on="recipe_id")

In [20]:
train.shape

(54427, 9)

In [21]:
test.shape

(19282, 9)

In [22]:
train

,date,rating,recipe_id,review,user_id,username,transformed_user_id,transformed_recipe_id,title
0,2012-09-11,5,222234,This recipe was delicious! Instead of cherry p...,3419993,Pie84,0,0,Cherry Folditup
1,2012-12-25,5,222234,Made this recipe as instructed and it was easy...,3154459,hertzen,1,0,Cherry Folditup
2,2012-11-22,5,222234,This was a great recipe to make using leftover...,10370475,suziloo,2,0,Cherry Folditup
3,2015-07-07,5,222234,"Tasty , simple dessert that comes out cute as ...",2304335,Chelsea M.,3,0,Cherry Folditup
4,2016-12-31,5,222234,"Delicious!! I was pressed for time, so I didn'...",1967176,Amanda H.,4,0,Cherry Folditup
...,...,...,...,...,...,...,...,...,...
54422,2015-10-05,2,239466,Cooking time inadequate. Should have been cove...,16243497,david,54505,1080,Chef John's Cranberry Bean Ragout
54423,2015-07-02,2,240133,I must have overcooked it.under whelming,6013253,janalee,54507,1082,Roast Quail with Cured Lemon
54424,2014-07-13,4,237724,This was easy to make and delicious. I subbed ...,8837240,CAN,54508,1083,Sausage-Stuffed Piquillo Peppers
54425,2013-01-04,5,222237,Thank you Chef John.I was looking for somethin...,2499050,gigithefoodie,54537,1112,Cotechino and Braised Beans


In [23]:
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')

In [24]:
train_summary = vectorizer.fit_transform(train.title).toarray()
test_summary = vectorizer.transform(test.title).toarray()

In [25]:
user_id_train = train.transformed_user_id[:-1000]
item_id_train = train.transformed_recipe_id[:-1000]
train_summary = train_summary[:-1000]
y_train = train.rating[:-1000]

user_id_test = test.transformed_user_id
item_id_test = test.transformed_recipe_id
y_test = test.rating

user_id_val = train.transformed_user_id[-1000:]
item_id_val = train.transformed_recipe_id[-1000:]
val_summary = train_summary[-1000:]
y_val = train.rating[-1000:]

In [26]:
train_summary.shape

(53427, 1059)

In [27]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

item_embedding_layer = layers.Embedding(input_dim = item_vocab_size + 1, output_dim=128, input_length = None)
item_id = keras.Input(shape=(1,), name='item_ids')
item_embeddings = item_embedding_layer(item_id)
item_embeddings = layers.Dropout(rate=0.3)(item_embeddings)
item_embeddings = layers.Flatten()(item_embeddings)
item_embeddings = keras.Model(inputs=item_id, outputs=item_embeddings)

user_id = keras.Input(shape=(1,), name='user_ids')
user_embeddings = layers.Embedding(input_dim = user_vocab_size + 1, output_dim=128, input_length = None)(user_id)
user_embeddings = layers.Dropout(rate=0.3)(user_embeddings)
user_embeddings = layers.Flatten()(user_embeddings)
user_embeddings = keras.Model(inputs=user_id, outputs=user_embeddings)

summary_inputs = keras.Input(shape=(train_summary.shape[1],), name='summary')
summ = layers.Dense(128, activation='relu', name='summary_dense_1')(summary_inputs)
summ = layers.Dropout(rate=0.2)(summ)
summ = layers.Dense(32, activation='relu', name='summary_dense_2')(summ)
summ = keras.Model(inputs=summary_inputs, outputs=summ)

combined = layers.concatenate([user_embeddings.output, item_embeddings.output, summ.output])
z = layers.BatchNormalization(name="bn_top_0")(combined)
z = layers.Dense(32, activation='relu', name='top_combined_dense_1')(z)
z = layers.Dropout(rate=0.2)(z)
z = layers.BatchNormalization(name="bn_top_1")(z)
z = layers.Dense(16, activation='relu', name='top_combined_dense_2')(z)
z = layers.Dense(1, name='predictions')(z)
hybrid_model = keras.Model(inputs=[user_id, item_id, summary_inputs], outputs=z)

In [28]:
hybrid_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),  # Optimizer
              # Loss function to minimize
              loss=tf.keras.losses.MSE,  
              # List of metrics to monitor
              metrics= [keras.metrics.MeanSquaredError()])

In [29]:
keras.backend.clear_session()
# del model
history = hybrid_model.fit([user_id_train, item_id_train, train_summary], y_train,
                    batch_size=1000,
                    epochs=20,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=([user_id_val, item_id_val, val_summary], y_val)
                   )
print('\nhistory dict:', history.history)

Epoch 1/20
54/54 [==============================] - 4s 75ms/step - loss: 13.8663 - mean_squared_error: 13.8663 - val_loss: 13.0458 - val_mean_squared_error: 13.0458
Epoch 2/20
54/54 [==============================] - 4s 71ms/step - loss: 3.1277 - mean_squared_error: 3.1277 - val_loss: 6.3570 - val_mean_squared_error: 6.3570
Epoch 3/20
54/54 [==============================] - 4s 71ms/step - loss: 1.1879 - mean_squared_error: 1.1879 - val_loss: 4.1016 - val_mean_squared_error: 4.1016
Epoch 4/20
54/54 [==============================] - 4s 71ms/step - loss: 0.8316 - mean_squared_error: 0.8316 - val_loss: 2.6110 - val_mean_squared_error: 2.6110
Epoch 5/20
54/54 [==============================] - 4s 71ms/step - loss: 0.6668 - mean_squared_error: 0.6668 - val_loss: 1.7184 - val_mean_squared_error: 1.7184
Epoch 6/20
54/54 [==============================] - 4s 71ms/step - loss: 0.5266 - mean_squared_error: 0.5266 - val_loss: 1.2442 - val_mean_squared_error: 1.2442
Epoch 7/20
54/54 [============

### Evaluation

#### Training metrics

In [30]:
predictions = hybrid_model.predict([user_id_train, item_id_train, train_summary])

In [31]:
print("train rmse:", math.sqrt(mean_squared_error(y_train, predictions)))

train rmse: 0.25392357373812124


#### Test metrics

In [32]:
predictions = hybrid_model.predict([user_id_test, item_id_test, test_summary])

In [33]:
print("test rmse:", math.sqrt(mean_squared_error(y_test, predictions)))

test rmse: 0.7529374208898267
